In [ ]:
import sys
sys.path.append('/home/hydrogen/workspace/Space_GW/pespace')
sys.path.append('/home/hydrogen/workspace/Space_GW/tiwave')
import taichi as ti
ti.init(arch=ti.cuda, default_fp=ti.f64, offline_cache=False)
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from tiwave.waveforms import IMRPhenomXAS


def print_struct_value(name, struct_field):
    print(f"{name}:")
    for key, data in struct_field.to_numpy().items():
        print(f"{key}: {data}")


[Taichi] version 1.7.2, llvm 15.0.4, commit 0131dce9, linux, python 3.10.14


[I 11/14/24 10:29:41.032 3648553] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Taichi] Starting on arch=cuda


In [9]:
params_in = dict(
    mass_1=159059.325070,
    mass_2=108817.001497,
    chi_1=-0.448101,
    chi_2=0.014488,
    luminosity_distance=7205.907347,
    inclination=2.123734,
    reference_phase=3.211444,
    coalescence_time=0.000000,
)

minimum_frequency = 1e-5
maximum_frequency = 0.1

cadence = 5
duration = 3600*24*30*12 # about one year
num_samples = int(duration/cadence)
full_freqs = np.fft.rfftfreq(num_samples, cadence)
freqs_mask = (full_freqs <= maximum_frequency) * (full_freqs >= minimum_frequency)
freqs = full_freqs[freqs_mask]
# print(full_freqs)
# print(full_freqs.shape)
# print(freqs)
# print(freqs.shape)

freqs_ti = ti.field(ti.f64, shape=freqs.shape)
freqs_ti.from_numpy(freqs)
wf = IMRPhenomXAS(freqs_ti)
wf.update_waveform(params_in)

`waveform_container` is not given, initializing one with returned_form=polarizations, include_tf=True


/home/hydrogen/workspace/Space_GW/tiwave/tiwave/waveforms/IMRPhenomX.py:2417: UserWarning: `parameter_sanity_check` is disable, make sure all parameters passed in are valid.
  warnings.warn(


TaichiSyntaxError: 
File "/home/hydrogen/workspace/Space_GW/tiwave/tiwave/waveforms/IMRPhenomX.py", line 2543, in _update_waveform_kernel:
        t_ffit = _compute_tf(
                 ^^^^^^^^^^^^
            powers_of_Mf,
            ^^^^^^^^^^^^^
            self.phase_coefficients[None],
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
            self.source_parameters[None],
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        )
        ^
Parameter `source_params : <taichi.types.annotations.Template object at 0x7fde7865e6b0>` missing. (<string>)

In [ ]:



@ti.kernel
def main():
    source_parameters[None].update_all_source_parameters(
        mass_1=params_in['mass_1'],
        mass_2=params_in['mass_2'],
        chi_1=params_in['chi_1'],
        chi_2=params_in['chi_2'],
        luminosity_distance=params_in['luminosity_distance'],
        inclination=params_in['inclination'],
        reference_phase=params_in['reference_phase'],
        coalescence_time=params_in['coalescence_time']
        )
    pn_prefactors[None].compute_PN_prefactors(source_parameters[None])
    amplitude_coefficients[None].compute_amplitude_coefficients(source_parameters[None], pn_prefactors[None])
    phase_coefficients[None].compute_phase_coefficients(source_parameters[None], pn_prefactors[None])

main()
# print_struct_value('source_parameters', source_parameters)
print_struct_value("PN prefactors", pn_prefactors)
# print_struct_value("amplitude coefficients", amplitude_coefficients)
print_struct_value("phase coefficients", phase_coefficients)


"""
prefactor_varphi_0: 0.14839418360069925
prefactor_varphi_1: 0.0
prefactor_varphi_2: 2.03338100344648
prefactor_varphi_3: -27.384742570935614
prefactor_varphi_4: 28.783459350352654
prefactor_varphi_5: 223.23078157609288
prefactor_varphi_5l: 223.23078157609288
prefactor_varphi_6: -2339.986759871465
prefactor_varphi_6l: -159.1988134289097
prefactor_varphi_7: 5089.122219660385
prefactor_varphi_8: -2644.245062170716
prefactor_varphi_8l: 2644.245062170716
prefactor_A_0: 1.0
prefactor_A_1: 0.0
prefactor_A_2: -1.7041022105523895
prefactor_A_3: -2.4575396595453576
prefactor_A_4: -21.57083898486485
prefactor_A_5: -23.872239534047463
prefactor_A_6: -43.28346143439896
phase coefficients:
sigma_1: -17535.3800126411
sigma_2: 346008.28733654856
sigma_3: -476763.4850222173
sigma_4: -1232666.902429353
ins_colloc_points: [0.0026     0.00664779 0.01474338 0.01879118]
ins_colloc_values: [17823.49583042 22473.84247014 20468.45581067 17592.64778799]
beta_0: -34.51183607684095
beta_1: -5.257534276252037
beta_2: 0.268588256993092
beta_3: -0.00443718336244198
beta_4: 3.44668099966693e-05
int_colloc_points: [0.017587   0.02185023 0.0321426  0.04243496 0.0466982 ]
int_colloc_values: [ 76.42573122   9.86906048 -44.09635589 -63.22200213 -67.83145753]
C1_int: 11.09200929827
C2_int: -6.114835553407431
useful_powers_fjoin_int_ins: {'third': array(0.26005421), 'two_thirds': array(0.06762819), 'one': array(0.017587), 'four_thirds': array(0.00457357), 'five_thirds': array(0.00118938), 'two': array(0.0003093), 'seven_thirds': array(8.04354009e-05), 'eight_thirds': array(2.09175647e-05), 'three': array(5.43970077e-06), 'four': array(9.5667997e-08), 'fourth': array(0.36416476), 'three_fourths': array(0.04829406), 'seven_sixths': array(0.00896858), 'log': array(-4.0405955)}
useful_powers_fjoin_MRD_int: {'third': array(0.36010851), 'two_thirds': array(0.12967814), 'one': array(0.0466982), 'four_thirds': array(0.01681642), 'five_thirds': array(0.00605574), 'two': array(0.00218072), 'seven_thirds': array(0.0007853), 'eight_thirds': array(0.00028279), 'three': array(0.00010184), 'four': array(4.75554843e-06), 'fourth': array(0.46486306), 'three_fourths': array(0.10045582), 'seven_sixths': array(0.02802314), 'log': array(-3.06404963)}
c_0: -45.080870324204064
c_1: -23.422597343913406
c_2: 0.12249109235063438
c_4: -5.504436614185911e-05
c_L: -0.003548862095733583
MRD_colloc_points: [0.04628034 0.05373884 0.07174527 0.07970038 0.09721021]
MRD_colloc_values: [-67.83145753 -75.41204912 -93.40576238 -99.67481614 -90.73364998]
C1_MRD: 17.290430205433864
C2_MRD: 15.608041981807752
"""
